In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import Model
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import Sequential
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input, Dropout, Activation, Dense, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from google.protobuf import text_format
import sklearn
import warnings
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
filepath = "/content/drive/MyDrive/iot_23_data/iot23_combined (2).csv"

In [ ]:
df = pd.read_csv(filepath)

In [ ]:
df

,Unnamed: 0,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,proto_tcp,proto_udp,conn_state_OTH,conn_state_REJ,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0,2.998796,0,0,0.0,3.0,180.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3,2.998804,0,0,0.0,3.0,180.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,4,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521502,99994,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1521503,99995,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1521504,99996,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1521505,99997,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
del df['Unnamed: 0']

In [ ]:
df['label'].value_counts()

8    880860
7    262690
0    240842
5    124362
1      8437
9      3915
2       357
4        30
6        13
3         1
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']].values

In [ ]:
X.shape

(1521507, 24)

In [ ]:
Y = pd.get_dummies(df['label']).values

In [ ]:
Y.shape

(1521507, 10)

In [ ]:
#X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
#Y = df[['label']]

In [ ]:
X

array([[2.998796, 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       ...,
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ]])

In [ ]:
df

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,proto_tcp,proto_udp,conn_state_OTH,conn_state_REJ,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,2.998796,0,0,0.0,3.0,180.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,2.998804,0,0,0.0,3.0,180.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521502,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1521503,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1521504,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1521505,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X)

MinMaxScaler()

In [ ]:
normalized_x = scaler.transform(X)

In [ ]:
normalized_x

array([[9.02120756e-05, 5.14615068e-05, 8.57558209e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.25598094e-05, 5.14615068e-05, 8.57558209e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.25598094e-05, 5.14615068e-05, 8.57558209e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.25598094e-05, 5.14615068e-05, 8.57558209e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.25598094e-05, 5.14615068e-05, 8.57558209e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.25598094e-05, 5.14615068e-05, 8.57558209e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
normalized_x.shape

(1521507, 24)

In [ ]:
scaler.fit(Y)

MinMaxScaler()

In [ ]:
normalized_y = scaler.transform(Y)

In [ ]:
normalized_y

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, random_state=10, test_size=0.2)

In [ ]:
X_train.shape

(1217205, 24)

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(2000, activation='relu',input_dim=24))
model.add(Dense(1500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(800,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(400,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(150,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2000)              50000     
                                                                 
 dense_1 (Dense)             (None, 1500)              3001500   
                                                                 
 dropout (Dropout)           (None, 1500)              0         
                                                                 
 dense_2 (Dense)             (None, 800)               1200800   
                                                                 
 dropout_1 (Dropout)         (None, 800)               0         
                                                                 
 dense_3 (Dense)             (None, 400)               320400    
                                                                 
 dropout_2 (Dropout)         (None, 400)               0

In [ ]:
import time

In [ ]:
start = time.time()
print('program start...')
print()

history = model.fit(X_train, Y_train, epochs = 10, batch_size=256, validation_data=(X_test,Y_test),verbose=1)

print('prediction:')
y_pred=model.predict(X_test)
print(y_pred)
print()

print()
end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

program start...

Epoch 1/10
4755/4755 [==============================] - 841s 177ms/step - loss: 0.8249 - accuracy: 0.7097 - val_loss: 0.8185 - val_accuracy: 0.7109
Epoch 2/10
4755/4755 [==============================] - 837s 176ms/step - loss: 0.8192 - accuracy: 0.7106 - val_loss: 0.8195 - val_accuracy: 0.7104
Epoch 3/10
4755/4755 [==============================] - 839s 176ms/step - loss: 0.8182 - accuracy: 0.7107 - val_loss: 0.8196 - val_accuracy: 0.7109
Epoch 4/10
4755/4755 [==============================] - 840s 177ms/step - loss: 0.8179 - accuracy: 0.7107 - val_loss: 0.8199 - val_accuracy: 0.7104
Epoch 5/10
4755/4755 [==============================] - 841s 177ms/step - loss: 0.8178 - accuracy: 0.7107 - val_loss: 0.8186 - val_accuracy: 0.7104
Epoch 6/10
4755/4755 [==============================] - 843s 177ms/step - loss: 0.8175 - accuracy: 0.7108 - val_loss: 0.8182 - val_accuracy: 0.7109
Epoch 7/10
4755/4755 [==============================] - 845s 178ms/step - loss: 0.8176 - accur

AttributeError: ignored

In [ ]:
y_pred

array([[4.22484096e-04, 3.99562580e-17, 1.37250940e-28, ...,
        1.21607238e-22, 1.44228260e-15, 1.60373756e-22],
       [1.05440721e-01, 6.41651312e-03, 1.30989632e-04, ...,
        1.97882786e-01, 6.70311391e-01, 3.03261254e-06],
       [1.05440721e-01, 6.41651312e-03, 1.30989632e-04, ...,
        1.97882786e-01, 6.70311391e-01, 3.03261254e-06],
       ...,
       [1.05440721e-01, 6.41651312e-03, 1.30989632e-04, ...,
        1.97882786e-01, 6.70311391e-01, 3.03261254e-06],
       [4.22484096e-04, 3.99564102e-17, 1.37248846e-28, ...,
        1.21607238e-22, 1.44228260e-15, 1.60373756e-22],
       [1.05440736e-01, 6.41651545e-03, 1.30989749e-04, ...,
        1.97882786e-01, 6.70311391e-01, 3.03261254e-06]], dtype=float32)

In [ ]:
model.evaluate(X_test,Y_test)

9510/9510 [==============================] - 112s 12ms/step - loss: 0.8179 - accuracy: 0.7104


[0.8178836703300476, 0.710425853729248]

In [ ]:
# y_pred=np.argmax(y_pred, axis=1)
# Y_test=np.argmax(Y_test, axis=1)

[[ 19477      0      0      0     10      0      0  28520      1]
 [     0      3      0      0      0      7      0   1769      0]
 [     0      0      0      0      0      1      0     75      0]
 [     0      0      0      1      0      0      0      7      0]
 [     0      0      0      0  20001      0      0   5071      0]
 [     0      0      0      0      0      0      0      0      0]
 [     1      0      0      0      0      0      0  52610      0]
 [     8      0      0      0      0      0      0 175944      0]
 [     0      0      0      0      0      0      0     38    758]]


In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))

Classifiction Report :
              precision    recall  f1-score   support

           0       1.00      0.41      0.58     48008
           1       1.00      0.00      0.00      1779
           2       0.00      0.00      0.00        76
           4       1.00      0.12      0.22         8
           5       1.00      0.80      0.89     25072
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00     52611
           8       0.67      1.00      0.80    175952
           9       1.00      0.95      0.97       796

    accuracy                           0.71    304302
   macro avg       0.63      0.36      0.38    304302
weighted avg       0.63      0.71      0.63    304302

